### Example: Equidistant geometry


In [ ]:
import os 
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import jax 
jax.config.update("jax_enable_x64", True)
import jax_sbgeom
import jax.numpy as jnp
%load_ext autoreload
%autoreload 2
import numpy as onp
import matplotlib.pyplot as plt
import pyvista as pv
from jax_sbgeom.jax_utils import mesh_to_pyvista_mesh

##### Setting up geometry

Selecting a particular coil and plasma set:

In [ ]:
stell_i = 2
vmec_file = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4",     "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias4_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4",     "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"][stell_i]
coil_file = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS4_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"][stell_i]

We use a FluxSurfaceConstantPhi, as this has the property that $\phi_{in} = \phi_{out}$ even beyond the LCFS (as required by FFTs). Furthermore, this creates a surface exactly the same as the original normal vector for constant $d$.

In [ ]:
from jax_sbgeom.flux_surfaces import FluxSurfaceNormalExtendedConstantPhi, ToroidalExtent
from jax_sbgeom.coils         import CoilSet, DiscreteCoil, convert_to_fourier_coilset, RotationMinimizedFrame, FiniteSizeCoilSet
flux_surface = FluxSurfaceNormalExtendedConstantPhi.from_hdf5(vmec_file)
def _get_discrete_coils(coil_file):
    import h5py
    with h5py.File(coil_file, 'r') as f:
        coil_data = jnp.array(f['Dataset1'])
    return CoilSet.from_list([DiscreteCoil.from_positions(coil_data[i]) for i in range(coil_data.shape[0])])
coilset      = _get_discrete_coils(coil_file)
fourier_coilset = convert_to_fourier_coilset(coilset)
fs_coilset      = FiniteSizeCoilSet.from_coilset(fourier_coilset, RotationMinimizedFrame, 100)

We load a HCPB blanket from a JSON file that describes layering and element compositions

In [ ]:
import json
with open("hcpb_blanket.json") as f:
    blanket_data = json.load(f)

In [ ]:
import jax_sbgeom.interfaces.blanket_creation as bc
fw_distance         = 0.2 
n_theta_blanket     = 55
n_phi_blanket       = 130
resolutions_blanket = [10, 1,1,6,4,3,4,3]  # Number of radial elements in each blanket layer for the tally mesh later
thicknesses         = [i['Thickness'] for i in blanket_data.values()]  # Thickness of each blanket layer
layers_jax          = jnp.concatenate([jnp.array([0.0]),jnp.cumsum(jnp.array(thicknesses))]) + fw_distance
discrete_blanket    = bc.LayeredDiscreteBlanket(tuple(layers_jax), n_theta_blanket, n_phi_blanket, tuple(resolutions_blanket), ToroidalExtent.full_module(flux_surface))
discrete_coilset    = bc.DiscreteFiniteSizeCoilSet(n_points_per_coil=500, toroidal_extent=ToroidalExtent.full_module(flux_surface), width_R=0.3, width_phi=0.3)

We then generate an equal-arclength representation of the all flux surface layers:

In [ ]:
from jax_sbgeom.flux_surfaces.convert_to_vmec import create_extended_flux_surface_d_interp_equal_arclength
fs_total = create_extended_flux_surface_d_interp_equal_arclength(flux_surface, jnp.array(discrete_blanket.d_layers),  flux_surface.settings.mpol * 6, flux_surface.settings.ntor * 6, 100)

We can mesh the geometry:

In [ ]:
surfaces = jax_sbgeom.flux_surfaces.mesh_watertight_layers(fs_total, 2.0 + jnp.arange(len(discrete_blanket.d_layers)), discrete_blanket.toroidal_extent, discrete_blanket.n_theta, discrete_blanket.n_phi)

coilset_trunc                    = jax_sbgeom.coils.coilset.filter_coilset_phi(fs_coilset, discrete_coilset.toroidal_extent.start, discrete_coilset.toroidal_extent.end)        
coil_vertices, coil_connectivity = jax_sbgeom.coils.mesh_coilset_surface(coilset_trunc, discrete_coilset.n_points_per_coil, discrete_coilset.width_R, discrete_coilset.width_phi)

In [ ]:
plotter = pv.Plotter()
colors = ["red", "orange", "yellow", "green", "blue", "indigo", "violet", "pink"]
for i, surface in enumerate(surfaces[1]):
    mesh = mesh_to_pyvista_mesh(surfaces[0], surface)
    plotter.add_mesh(mesh, color=colors[i%len(colors)], opacity=0.5, show_edges=True)

plotter.add_mesh(mesh_to_pyvista_mesh(coil_vertices, coil_connectivity), show_edges=True)
plotter.show()

### Coupling to OpenMC

In [ ]:
blanket_data.keys()

In [ ]:
import openmc

In [ ]:
def create_openmc_material_from_elements(material_dict, name):
    elements_i = material_dict["Elements"]
    mat_i = openmc.Material(name = name)        
    total_atom_number = sum(elements_i.values())
    for atom in elements_i.keys():
        nuclide_name = atom.title()
        if atom.title() == "D":
            nuclide_name = "H2"
        mat_i.add_nuclide(nuclide_name , elements_i[atom] / total_atom_number, 'ao')
    mat_i.set_density("atom/cm3", float(total_atom_number) * 1e-6) # m3 -> cm3
    return mat_i

In [ ]:
total_material_dict = {}
for i in blanket_data.keys():    
    total_material_dict[i] = create_openmc_material_from_elements(blanket_data[i], i)
# Add coil material (same as BackSupportingStructure)    
total_material_dict['Coil'] = create_openmc_material_from_elements(blanket_data['BackSupportingStructure'], 'Coil')

In [ ]:
names = list(blanket_data.keys())
dagmc_surface_model = jax_sbgeom.interfaces.dagmc_interface.create_dagmc_surface_mesh(discrete_blanket, fs_total, names, fs_coilset, discrete_coilset, 'Coil')
dagmc_surface_model_filename = "dagmc_helias5_blanket.h5m"
dagmc_surface_model.write_file(dagmc_surface_model_filename)

In [ ]:
import openmc 

In [ ]:
dagmc_univ = openmc.DAGMCUniverse(filename  =  dagmc_surface_model_filename).bounded_universe()    
geometry   = openmc.Geometry(root = dagmc_univ)
materials  = openmc.Materials(total_material_dict.values())
geometry.export_to_xml()
materials.export_to_xml()

We can plot the OpenMC geometry:

In [ ]:
plot = openmc.Plot.from_geometry(geometry, basis='xy', slice_coord =0)
plot.color_by = 'material'

In [ ]:
materials = openmc.Materials(total_material_dict.values())
geometry.export_to_xml()
materials.export_to_xml()

In [ ]:
openmc.plot_inline(plot)

We define a plasma source derived from some simple parametric values:

In [ ]:
n0 = 2.4e20
Ti0=  15.2
nalpha = 0.35
Tialpha = 1.2
s = jnp.linspace(0, 1, 100)
fs_rates = jax_sbgeom.interfaces.plasma.flux_surface_reaction_rates_simple(s, n0, nalpha, Ti0, Tialpha)

In [ ]:
plt.plot(jnp.sqrt(s), fs_rates)
plt.xlabel("$\\rho = \\sqrt\{s\}$")
plt.ylabel("Neutron source density [m^{-3} s^{-1}]")

Then, we create a tetrahedral mesh for the geometry:

In [ ]:
n_plasma_source = 10
tetrahedral_plasma_mesh = jax_sbgeom.flux_surfaces.mesh_tetrahedra(flux_surface, jnp.linspace(0,1,10) **2, discrete_blanket.toroidal_extent, discrete_blanket.n_theta, discrete_blanket.n_phi)

We need to map that to the source values by using that the first axis of the mesh points is the layering:

In [ ]:
s_axis = jnp.zeros(discrete_blanket.n_phi)
s_layers = jnp.broadcast_to((jnp.linspace(0,1, n_plasma_source)[1:][:, None])**2, (n_plasma_source - 1, discrete_blanket.n_phi *  discrete_blanket.n_theta)).ravel()
s_total = jnp.concatenate([s_axis, s_layers])
neutron_rates_mesh = jax_sbgeom.interfaces.plasma.flux_surface_reaction_rates_simple(s_total, n0, nalpha, Ti0, Tialpha)

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(mesh_to_pyvista_mesh(tetrahedral_plasma_mesh), scalars=neutron_rates_mesh, show_edges=True)
plotter.show()

Then, we average it for each tetrahedron:

In [ ]:
tet_volumes = jax_sbgeom.flux_surfaces.flux_surface_meshing._volumes_tetrahedra(*tetrahedral_plasma_mesh)
rates_element = jnp.mean(neutron_rates_mesh[tetrahedral_plasma_mesh[1]], axis=-1)

Creating all the independent sources for the mesh source using the OpenMC functions:

In [ ]:
dagmc_source_file_mesh = "dagmc_helias5_plasma_source.h5m"
jax_sbgeom.interfaces.dagmc_interface.tetrahedral_mesh_to_moab_mesh(*tetrahedral_plasma_mesh).write_file(dagmc_source_file_mesh)


centre_tetrahedral_plasma_mesh = [float(i * 100.0) for i in onp.mean(tetrahedral_plasma_mesh[0], axis=0)]
def base_source(strength):
    base_source = openmc.IndependentSource()
    # this will be ignored when sampling. However, it still gets checked that the source is inside the geometry so we have to give some valid point inside
    base_source.space = openmc.stats.Point(centre_tetrahedral_plasma_mesh) 
    base_source.angle  = openmc.stats.Isotropic()
    base_source.energy = openmc.stats.Discrete([14.1e6], [1.0])  # 14.1 MeV neutrons
    base_source.strength = strength
    return base_source

reaction_rates_volume_weighted = onp.array(rates_element * tet_volumes)

openmc_mesh_sources = [base_source(float(rate)) for rate in reaction_rates_volume_weighted]


In [ ]:
umesh_source = openmc.UnstructuredMesh(filename=dagmc_source_file_mesh, library="moab", name = 'PlasmaSourceMesh')
meshsource  = openmc.MeshSource(umesh_source, openmc_mesh_sources)

Settings use a relatively low 1 million particles:

In [ ]:
settings = openmc.Settings()

settings.batches = 100
settings.particles = 10000   
settings.run_mode = 'fixed source'
settings.source = meshsource
settings.output = {'tallies': False, 'summary': False}
settings.export_to_xml()

In [ ]:
tally_mesh_filename = "dagmc_helias5_tally_mesh.h5m"

tally_mesh_tetra = jax_sbgeom.interfaces.blanket_creation.mesh_tetrahedral_blanket(fs_total, discrete_blanket, s_power_sampling=2)
jax_sbgeom.interfaces.dagmc_interface.tetrahedral_mesh_to_moab_mesh(*tally_mesh_tetra).write_file(tally_mesh_filename)

umesh_tally = openmc.UnstructuredMesh(filename=tally_mesh_filename, library="moab", name = 'TallyMesh')
mesh_filter = openmc.MeshFilter(umesh_tally)

tally_flux = openmc.Tally(name='FluxTally')
tally_flux.filters = [mesh_filter]
tally_flux.scores = ['flux']
tally_flux.estimator = "tracklength"
openmc.Tallies([tally_flux]).export_to_xml()

In [ ]:
os.environ["OPENMC_CROSS_SECTIONS"] = str("/home/tbogaarts/stellarator_paper/base_data/xs" + "/cross_sections.xml")
# preset data file.

In [ ]:
openmc.run()

In [ ]:

def Get_OpenMC_Flux(statepoint , tally_id : int = 1, value = 'mean', meter_exponent = -2):
    '''
    Get the flux from an OpenMC statepoint file.
    Uses a 1D mesh because the unstructured mesh is by definition in a 1D format.
    Parameters
    ----------
    statepoint : str, PathLike
        The path to the OpenMC statepoint file.
    tally_id : int, optional
        The ID of the tally to extract the flux from. Defaults to 1.
    Returns
    -------
    np.ndarray
        The scalar flux in units of m^{-2} s^{-1} (scaled by 1e4 with respect to OpenMC and the unstructured mesh volumes).

    '''
    with openmc.StatePoint(statepoint) as sp:
        sp_tally = sp.get_tally(id = tally_id)
        sp_umesh = sp_tally.find_filter(openmc.MeshFilter).mesh
        if value == "mean" or value == "std_dev":
            return sp_tally.get_reshaped_data(value =value)[:,  0, 0] * (1e-2)** meter_exponent / onp.abs(sp_umesh.volumes[:]) # scaled by 1e4:
        else:
            return onp.flip(sp_tally.get_reshaped_data(value =value)[:,  0, 0], axis = -1)
        
flux = Get_OpenMC_Flux("statepoint.100.h5", tally_flux.id)


In [ ]:
layer_flux = onp.zeros(discrete_blanket.n_discrete_layers)
for i in range(discrete_blanket.n_discrete_layers):
    layer_flux[i] = jnp.mean(flux[discrete_blanket.layer_slice(i)])


In [ ]:
discrete_blanket.n_discrete_layers

In [ ]:
x_values = jax_sbgeom.interfaces.blanket_creation.compute_d_spacing(discrete_blanket, 2)

In [ ]:
plt.stairs(layer_flux, x_values)
plt.yscale('log')
plt.xlim(1, discrete_blanket.d_layers[-1] + 1.0)
plt.ylim()
plt.xlabel("Distance from LCFS [m]")
plt.ylabel("Flux [m$^{-2}$ s$^{-1}$]")

In [ ]:
plotter = pv.Plotter()

plotter.add_mesh(mesh_to_pyvista_mesh(tally_mesh_tetra[0], tally_mesh_tetra[1][discrete_blanket.layer_slice(15)]), scalars=flux[discrete_blanket.layer_slice(15)], show_edges=True)
plotter.show()

In [ ]:
plotter = pv.Plotter()
fluxlog10 = onp.log10(flux)

safe_fluxlog10 = onp.where(fluxlog10 < -5000, onp.nan, fluxlog10)
plotter.add_mesh(mesh_to_pyvista_mesh(tally_mesh_tetra), scalars=safe_fluxlog10, show_edges=True)
plotter.show()